In [1]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate, remove_bad_data, chi2_analysis
import pandas as pd
import numpy as np
# from tqdm import tqdm

from scipy import stats
print(pp.__version__)

2.0.1


In [2]:
def create_measurement_unit(df_measurement, net):
    list_value=[]
    list_std=[]
    for index, row in df_measurement.iterrows():
        if row['element_type'] =='bus':
            if row['meas_type'] =='v':
                mu= net.res_bus.iloc[row['element'],0]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
            elif row['meas_type'] =='p':
                mu= net.res_bus.iloc[row['element'],2]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
            elif row['meas_type'] =='q':
                mu= net.res_bus.iloc[row['element'],3]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
                
        elif row['element_type'] =='line':
            if row['side']=='from':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],0]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],1]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],6]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
            elif row['side']=='to':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],2]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],3]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],7]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                    
                    
        elif row['element_type'] =='trafo':
            if row['side']=='from':
                if row['meas_type'] == 'v':
                    mu =net.res_trafo.iloc[row['element'],8]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] == 'p':
                    mu =net.res_trafo.iloc[row['element'],0]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] == 'q':
                    mu =net.res_trafo.iloc[row['element'],1]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
            elif row['side']=='to':
                if row['meas_type'] =='v':
                    mu =net.res_trafo.iloc[row['element'],10]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] =='p':
                    mu =net.res_trafo.iloc[row['element'],2]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] =='q':
                    mu =net.res_trafo.iloc[row['element'],3]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
#         print(mu)  
        value = np.random.normal(mu, sigma, 1)
        list_value.append(value[0])
        list_std.append(sigma)
    df_measurement['value']=list_value
    df_measurement['std_dev']=list_std

    for index, row in df_measurement.iterrows():
        if row['element_type']=='bus':
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'])
        elif row['element_type']=='line':
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])
        elif row['element_type']=='trafo':
            if row['meas_type'] in ['p','q']:
                pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                      std_dev=row['std_dev'], element=row['element'], side=row['side'])
    return df_measurement, net

In [104]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.028256,0.013475,-0.027611,-0.013328,0.000644,0.000147,0.001807,0.044405,1.0,0.0,0.996608,-0.208127,12.521762


# solar add in bus 3

In [119]:
net = pn.simple_four_bus_system()
# print(net4.sgen)
net.sgen['p_mw']=[0,0.015]
net.sgen['q_mvar']=[0,0.002]
net.sgen['sn_mva']=[0,0.02]
print(net.sgen)

pp.runpp(net)

  name  bus   p_mw  q_mvar  sn_mva  scaling  in_service  type  current_source
0  pv1    2  0.000   0.000    0.00      1.0        True  None            True
1  pv2    3  0.015   0.002    0.02      1.0        True  None            True


## Solar PV added in both bus 2,3

In [120]:
net = pn.simple_four_bus_system()
pp.runpp(net)

In [121]:
print(net.res_trafo)

    p_hv_mw  q_hv_mvar   p_lv_mw  q_lv_mvar     pl_mw   ql_mvar   i_hv_ka  \
0  0.028256   0.013475 -0.027611  -0.013328  0.000644  0.000147  0.001807   

    i_lv_ka  vm_hv_pu  va_hv_degree  vm_lv_pu  va_lv_degree  loading_percent  
0  0.044405       1.0           0.0  0.996608     -0.208127        12.521762  


In [124]:
df_measurement

,meas_type,element_type,element,side,value,std_dev
0,v,bus,1,None,0.969821,0.014949
1,v,bus,2,None,0.937605,0.014066
2,v,bus,3,None,0.930029,0.013530
3,p,line,0,from,0.028733,0.000690
4,p,line,1,from,0.015740,0.000393
5,q,line,0,from,0.013072,0.000333
6,q,line,1,from,0.008269,0.000202
7,i,line,0,from,0.045278,0.001110
8,i,line,1,from,0.027625,0.000680
9,p,trafo,0,from,0.028179,0.000706


In [123]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','p','p','q','q','i','i','p','q','v' ]
df_measurement['element_type']=['bus','bus','bus','line','line','line','line','line','line','trafo','trafo','trafo']
df_measurement['element']=[1,2,3,0,1,0,1,0,1,0,0,0]
df_measurement['side']=['None','None','None','from','from','from','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

#generate measurement data with error
df_measurement, net = create_measurement_unit(df_measurement, net)


#State Estimation 
success = estimate(net, init="flat", calculate_voltage_angles=True, zero_injection=[1])
V_est, delta_est = net.res_bus_est.vm_pu, net.res_bus_est.va_degree

In [125]:
net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.997159,0.000000,0.027536,0.012637
1,1.000440,-149.790840,-0.056824,-0.025853
2,0.939795,-148.644921,0.010943,0.004551
3,0.904063,-147.800546,0.014998,0.008182


In [ ]:
net.res_

In [126]:
print(net.load)

    name  bus  p_mw  q_mvar  const_z_percent  const_i_percent  sn_mva  \
0  load1    2  0.03    0.01              0.0              0.0     NaN   
1  load2    3  0.03    0.01              0.0              0.0     NaN   

   scaling  in_service  type  
0      1.0        True  None  
1      1.0        True  None  


In [127]:
print(net.sgen)

  name  bus   p_mw  q_mvar  sn_mva  scaling  in_service  type  current_source
0  pv1    2  0.020   0.005    0.03      1.0        True  None            True
1  pv2    3  0.015   0.002    0.02      1.0        True  None            True


## load reduction at both bus 2,3

In [129]:
net = pn.simple_four_bus_system()
net.load['p_mw']=[0.01, 0.015]
net.load['q_mvar']=[0.005, 0.008]

net.sgen['p_mw']=[0,0]
net.sgen['q_mvar']=[0,0]
net.sgen['sn_mva']=[0,0]
# here we have net p,q in bus 2,3
pp.runpp(net)


In [130]:
# df_measurement=pd.DataFrame()
# df_measurement=pd.DataFrame()
# df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
# df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
# df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
# df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']


df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','p','p','q','q','i','i','p','q','v' ]
df_measurement['element_type']=['bus','bus','bus','line','line','line','line','line','line','trafo','trafo','trafo']
df_measurement['element']=[1,2,3,0,1,0,1,0,1,0,0,0]
df_measurement['side']=['None','None','None','from','from','from','from','from','from','from','from','from']


#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

#generate measurement data with error
df_measurement, net = create_measurement_unit(df_measurement, net)


#State Estimation 
success = estimate(net, init="flat", calculate_voltage_angles=True, zero_injection=[1])
V_est, delta_est = net.res_bus_est.vm_pu, net.res_bus_est.va_degree

In [131]:
net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.997016,0.000000,0.027558,0.013879
1,1.000489,-149.794050,-0.056096,-0.027132
2,0.941350,-148.634315,0.010383,0.005117
3,0.906037,-147.861840,0.014919,0.007662


In [132]:
net.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.027893,0.013105,-0.025989,-0.012864,0.001904,0.000241,0.044460,0.044463,0.044463,1.000489,-149.794050,0.941350,-148.634315,31.312313
1,0.015606,0.007747,-0.014919,-0.007662,0.000687,0.000084,0.026715,0.026718,0.026718,0.941350,-148.634315,0.906037,-147.861840,18.815725


In [133]:
net.res_trafo_est

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,-0.027558,-0.013879,0.028203,0.014027,0.000645,0.000149,0.001787,0.045443,0.997016,0.0,1.000489,-149.79405,12.593491


In [134]:
print(net.res_ext_grid)

       p_mw    q_mvar
0  0.028256  0.013475


In [221]:
net1 = pn.simple_four_bus_system()
pp.runpp(net1)

net2 = pn.simple_four_bus_system()
net2.load['p_mw']=[0.01, 0.015]
net2.load['q_mvar']=[0.005, 0.008]
# net2.sgen['p_mw']=[0,0]
# net2.sgen['q_mvar']=[0,0]
# net2.sgen['sn_mva']=[0,0]
net2.sgen['scaling']=[0,0]
# here we have net p,q in bus 2,3
pp.runpp(net2)



In [222]:
net2.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source
0,pv1,2,0.020,0.005,0.03,0,True,None,True
1,pv2,3,0.015,0.002,0.02,0,True,None,True


In [223]:
net1.res_bus-net2.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,0.0,0.000000e+00,0.000000e+00,0.0
1,0.0,1.665335e-16,0.000000e+00,0.0
2,0.0,2.220446e-16,-1.734723e-18,0.0
3,0.0,6.661338e-16,-5.204170e-18,0.0


In [224]:
net1.res_line-net2.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,1.665335e-16,0.0,2.220446e-16,0.000000e+00
1,0.0,0.0,0.0,-1.214306e-17,0.0,-1.214306e-17,0.0,1.040834e-17,1.040834e-17,0.0,2.220446e-16,0.0,6.661338e-16,3.552714e-15


In [225]:
net1.res_trafo-net2.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.665335e-16,0.0


In [226]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','p','p','q','q','i','i','p','q','v' ]
df_measurement['element_type']=['bus','bus','bus','line','line','line','line','line','line','trafo','trafo','trafo']
df_measurement['element']=[1,2,3,0,1,0,1,0,1,0,0,0]
df_measurement['side']=['None','None','None','from','from','from','from','from','from','from','from','from']
#where bus measurement has 3%error

# df_measurement, net2 = create_measurement_unit(df_measurement, net2)
# success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])


In [227]:
df_measurement, net1 , net2= create_measurement_unit(df_measurement, net1)
success = estimate(net1, init="flat", calculate_voltage_angles=True, zero_injection=[1])
success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])




In [228]:
net1.res_bus_est-net2.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


In [234]:
net1.res_line_est-net2.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [231]:
net1.res_trafo_est-net2.res_trafo_est

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [232]:
net2.res_bus-net2.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.001648,0.000000,-0.054992,-0.026915
1,-0.005105,149.592420,0.054865,0.026939
2,-0.005650,149.599789,0.000396,-0.000060
3,-0.005229,149.617780,-0.000288,0.000025


In [233]:
net1.res_bus-net1.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.001648,0.000000,-0.054992,-0.026915
1,-0.005105,149.592420,0.054865,0.026939
2,-0.005650,149.599789,0.000396,-0.000060
3,-0.005229,149.617780,-0.000288,0.000025


In [284]:
net1 = pn.simple_four_bus_system()
pp.runpp(net1)

net2 = pn.simple_four_bus_system()
net2.load['p_mw']=[0.03, 0.015]
net2.load['q_mvar']=[0.01, 0.008]
net2.sgen['scaling']=[1,0]
# here we have net p,q in bus 2,3
pp.runpp(net2)




In [285]:
net1.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,load1,2,0.03,0.01,0.0,0.0,NaN,1.0,True,None
1,load2,3,0.03,0.01,0.0,0.0,NaN,1.0,True,None


In [286]:
net2.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,load1,2,0.030,0.010,0.0,0.0,NaN,1.0,True,None
1,load2,3,0.015,0.008,0.0,0.0,NaN,1.0,True,None


In [287]:
net1.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source
0,pv1,2,0.020,0.005,0.03,1.0,True,None,True
1,pv2,3,0.015,0.002,0.02,1.0,True,None,True


In [288]:
net2.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source
0,pv1,2,0.020,0.005,0.03,1,True,None,True
1,pv2,3,0.015,0.002,0.02,0,True,None,True


In [289]:
net1.res_bus-net2.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,0.0,0.000000e+00,0.000000e+00,0.0
1,0.0,2.775558e-17,0.000000e+00,0.0
2,0.0,2.220446e-16,0.000000e+00,0.0
3,0.0,2.220446e-16,-3.469447e-18,0.0


In [290]:
net1.res_line-net2.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,2.775558e-17,0.0,2.220446e-16,0.000000e+00
1,0.0,0.0,0.0,-5.204170e-18,0.0,-5.204170e-18,0.0,3.469447e-18,3.469447e-18,0.0,2.220446e-16,0.0,2.220446e-16,3.552714e-15


In [292]:
net1.res_ext_grid-net2.res_ext_grid

,p_mw,q_mvar
0,0.0,0.0


In [293]:
net1.res_trafo-net2.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.775558e-17,0.0


In [294]:
net1

This pandapower network includes the following parameter tables:
   - bus (4 elements)
   - load (2 elements)
   - sgen (2 elements)
   - ext_grid (1 element)
   - line (2 elements)
   - trafo (1 element)
   - bus_geodata (4 elements)
 and the following results tables:
   - res_bus (4 elements)
   - res_line (2 elements)
   - res_trafo (1 element)
   - res_ext_grid (1 element)
   - res_load (2 elements)
   - res_sgen (2 elements)

In [280]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','p','p','q','q','i','i','p','q','v' ]
df_measurement['element_type']=['bus','bus','bus','line','line','line','line','line','line','trafo','trafo','trafo']
df_measurement['element']=[1,2,3,0,1,0,1,0,1,0,0,0]
df_measurement['side']=['None','None','None','from','from','from','from','from','from','from','from','from']
#where bus measurement has 3%error

# df_measurement, net2 = create_measurement_unit(df_measurement, net2)
# success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])



In [281]:
df_measurement, net1 , net2= create_measurement_unit(df_measurement, net1)
success = estimate(net1, init="flat", calculate_voltage_angles=True, zero_injection=[1])
success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])

In [282]:
net2.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.996202,0.000000,0.028820,0.013368
1,0.999660,-149.781218,-0.056886,-0.027243
2,0.941295,-148.538139,0.009770,0.005040
3,0.905480,-147.674622,0.015037,0.008349


In [283]:
net1.res_bus_est-net2.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0


# Test

In [3]:
net = pn.simple_four_bus_system()
print(net.sgen)
print(net.load)
# here we have net p,q in bus 2,3
net.sgen['scaling']=[1, 1]
print(net.sgen)
pp.runpp(net)

  name  bus   p_mw  q_mvar  sn_mva  scaling  in_service  type  current_source
0  pv1    2  0.020   0.005    0.03      1.0        True  None            True
1  pv2    3  0.015   0.002    0.02      1.0        True  None            True
    name  bus  p_mw  q_mvar  const_z_percent  const_i_percent  sn_mva  \
0  load1    2  0.03    0.01              0.0              0.0     NaN   
1  load2    3  0.03    0.01              0.0              0.0     NaN   

   scaling  in_service  type  
0      1.0        True  None  
1      1.0        True  None  
  name  bus   p_mw  q_mvar  sn_mva  scaling  in_service  type  current_source
0  pv1    2  0.020   0.005    0.03        1        True  None            True
1  pv2    3  0.015   0.002    0.02        1        True  None            True


In [4]:
global upper_bus_accuracy, lower_bus_accuracy, upper_line_accuracy, lower_line_accuracy, upper_trafo_accuracy, lower_trafo_accuracy
upper_bus_accuracy=1.03
lower_bus_accuracy=0.97
upper_line_accuracy=1.05
lower_line_accuracy=0.95
upper_trafo_accuracy=1.05
lower_trafo_accuracy=0.95

In [5]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','line','line','line','line','line','line']
df_measurement['element']=[2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','from','from','from','from','from','from']
#where bus measurement has 3%error


df_measurement, net = create_measurement_unit(df_measurement, net)
success = estimate(net, init="slack", calculate_voltage_angles=True, zero_injection=[1])

we are here


In [6]:
print(net.res_bus_est)

      vm_pu   va_degree          p_mw        q_mvar
0  1.006505    0.000000 -2.798582e-02 -1.312183e-02
1  1.003201 -150.204009 -2.043762e-15 -9.135937e-16
2  0.945366 -149.058661  9.753767e-03  4.657016e-03
3  0.909799 -148.249027  1.505176e-02  8.005889e-03


In [7]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.027611,0.013328,-0.025713,-0.013088,0.001899,0.000241,0.044405,0.044408,0.044408,0.996608,-0.208127,0.93776,0.992541,31.273099
1,0.015713,0.008088,-0.015000,-0.008000,0.000713,0.000088,0.027200,0.027203,0.027203,0.937760,0.992541,0.90200,1.815913,19.157250


In [8]:
sum(net.res_bus_est.p_mw)

-0.003180292893098199

In [846]:
df_pseudo_load=pd.DataFrame()
df_pseudo_load['meas_type']=['p','q','p','q']
df_pseudo_load['element_type']=['bus','bus','bus','bus']
df_pseudo_load['element']=[2,2,3,3]
df_pseudo_load['side']=['None','None','None','None']

df_pseudo_load['value']=[-1.061068e-02,  -5.624722e-03 , -1.524357e-02,-8.017893e-03]
# df_pseudo_load['value']=[-0.031992,  -0.009605 , -0.027702+0.00252,-0.010058]
# df_pseudo_load['value']=[-0.0110711+0.001201,  -0.004941, -0.029891,-0.009584]
# df_pseudo_load['value']=[-0.0110711,  -0.004941, -0.029891+0.00106,-0.009584]

for index, row in df_pseudo_load.iterrows():
    if row['element_type']=='bus':
        pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                              std_dev=0.0001, element=row['element'])

success = estimate(net, init="results", calculate_voltage_angles=True, zero_injection=[1])
print(net.res_bus_est)

      vm_pu   va_degree          p_mw        q_mvar
0  0.983226    0.000000 -2.924867e-02 -1.398503e-02
1  0.979649 -150.222987  5.418723e-14 -1.761502e-14
2  0.917613 -148.928259  1.058355e-02  5.508792e-03
3  0.880619 -148.076391  1.516561e-02  7.950907e-03


In [847]:
net.measurement

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,2,0.901632,0.014066,None
1,None,v,bus,3,0.890311,0.013530,None
2,None,p,line,0,0.027514,0.000690,from
3,None,p,line,1,0.015237,0.000393,from
4,None,q,line,0,0.012557,0.000333,from
5,None,q,line,1,0.008256,0.000202,from
6,None,i,line,0,0.044353,0.001110,from
7,None,i,line,1,0.026941,0.000680,from
8,None,p,bus,2,-0.010611,0.000100,None
9,None,q,bus,2,-0.005625,0.000100,None


In [813]:
net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,1.002354,0.000000,-2.865328e-02,-1.348173e-02
1,0.998950,-150.210588,-2.367889e-15,2.350465e-15
2,0.939446,-149.020658,1.048794e-02,5.099151e-03
3,0.904051,-148.211885,1.488602e-02,7.902103e-03


In [399]:
net.measurement

,name,measurement_type,element_type,element,value,std_dev,side
0,None,v,bus,1,0.994124,0.014892,None
1,None,v,bus,2,0.821346,0.012493,None
2,None,v,bus,3,0.741309,0.011233,None
3,None,p,line,0,0.077621,0.001912,from
4,None,p,line,1,0.034916,0.000839,from
5,None,q,line,0,0.021584,0.000553,from
6,None,q,line,1,0.010020,0.000261,from
7,None,i,line,0,0.113187,0.002894,from
8,None,i,line,1,0.061581,0.001524,from
9,None,p,bus,0,0.078963,0.001161,None


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-0.028256,-0.013475
1,0.996608,-0.208127,0.000000,0.000000
2,0.937760,0.992541,0.010000,0.005000
3,0.902000,1.815913,0.015000,0.008000


,p_mw,q_mvar
0,0.020,0.005
1,0.015,0.002
